# KI - Projekt: Evidenztheorie

Anhand von Sensor-Daten soll kassifiziert werden, ob es sich bei dem vermessenen Objekt um einen PKW, einen LKW, ein Motorrad oder ein Fahrrad handelt.
<br>
Folgende Kategorien werden unterschieden: <br>

| <b>KFZ-Art<b> | <b>Geschwindigkeit</b> | <b>Beschleunigung</b>| <b>Höhe</b> | <b>Breite</b> |
| --------------|------------------------|----------------------|-------------|-------------- |
| <b>Fahrrad</b> | Langsam | - | hoch | schmal |
| <b>PKW</b> | Mittel | - | flach | breit |
| <b>LKW</b> | Mittel | - | hoch | breit |
| <b>Motorrad</b> | Mittel | Stark | hoch | schmal |

- Mittlere Geschwindigkeit zwischen 50 und 90 km/h.

## Schritte / Methoden:
1. Daten einlesen und explorieren.
2. Grenzwerte für 'hoch', 'flach', 'starke Beschleunigung', ... berechnen.
3. Einzelne Datensequenzen einlesen und Fahrzeugart mithilfe von `python_evidence`-Framework bestimmen.
  1. Werte (Geschw., Beschl., ...) bestimmen.
  2. Evidenz-Messung (+ Wahrscheinlichkeiten) festlegen.
  3. Fahrzeugart bestimmen.

## Hinweis
- Sensoren arbeiten auf hohen Entfernungen ungenauer:
  - Mögliche Fehlerquelle, möglicherweise korrigieren / einberechnen.

In [7]:
# Imports
import pandas as pd
from pathlib import Path

from IPython.display import display

from python_evidence.dempster_handler import DempsterHandler
from utils import load_data, change_data, get_velocity, get_acceleration

# Schritt 1: Daten einlesen und explorieren

--------------------------------------------------

Daten befinden sich in Form von CSV Dateien im [data](/data)-Ordner.

Sie bestehen aus jeweils einzelnen Mess-Sequenzen für ein Fahrzeug. Die Messdaten bestehen aus:
- t
  - Zeitpunkte (Messung alle 100ms)
- Eigengeschwindigkeit(m/s)
- Bbox_Breite
- Bbox_Höhe
- Bbox_links_unten_y
- Abstand(m)

In [8]:
"""
TODO:
    * Daten laden / explorieren.

    * Geschwindigkeiten von m/s in km/h.
    * Beschleunigung.
    * Höhe / Breite.
    
    * Grenzwerte berechnen.

    * Evidenztheorie für einzelne CSV Dateien.
"""

DATA_FOLDER = Path("data")
CSV_FILES = ["DPE15_1.csv", "DPE15_2.csv", "DPE15_3.csv", "DPE15_4.csv", 
             "DPE16_1.csv", "DPE16_2.csv", "DPE16_3.csv", "DPE16_4.csv"]

for ix, path in enumerate(CSV_FILES):
    CSV_FILES[ix] = DATA_FOLDER.joinpath(path)

# Daten laden.
data_dict = load_data(CSV_FILES)

# Datensatz explorieren.
df = data_dict["DPE15_1"]
display(df)

print(df.info())

"""
TODO:
    * Eigengeschwindigkeit zu floats
    * Gechwindigkeit m/s in km/h.
    * Abstand(m) zu floats.
    * Beschleunigung.
    * Höhe / Breite.
"""
change_data(data_dict)

# print(data_dict["DPE15_1"].info())
# display(df)

# Geschwindigkeiten / Acceleration vom anderen Fahrzeug
for path, df in data_dict.items():
    print(path)
    print("--------------------------")
    get_velocity(df)
    get_acceleration(df)
    print("\n\n")

print(data_dict["DPE16_2"].info())
display(data_dict["DPE16_2"])

# Save updated CSV files
for path, df in data_dict.items():
    p = Path("updated_data").joinpath(F"{path}.csv")
    df.to_csv(p, sep=";")


,t,Eigengeschwindigkeit(m/s),Bbox_Breite,Bbox_Höhe,Bbox_links_unten_y,Abstand(m)
0,0,"16,1",61,121,142,71
1,"0,1","16,1",64,128,141,"70,5"
2,"0,2",16,64,127,138,69
3,"0,3","16,1",66,131,138,69
4,"0,4",16,67,134,137,"68,5"
...,...,...,...,...,...,...
99,"9,9","16,1",181,362,22,11
100,10,"16,2",183,366,20,10
101,"10,1","16,1",183,365,18,9
102,"10,2","16,1",230,374,18,9


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   t                          104 non-null    object
 1   Eigengeschwindigkeit(m/s)  104 non-null    object
 2   Bbox_Breite                104 non-null    int64 
 3   Bbox_Höhe                  104 non-null    int64 
 4   Bbox_links_unten_y         104 non-null    int64 
 5   Abstand(m)                 104 non-null    object
dtypes: int64(3), object(3)
memory usage: 5.0+ KB
None
DPE15_1
--------------------------
11.1 11.1 0.0
1.0 11.1 -10.1
16.1 1.0 15.100000000000001
11.0 16.1 -5.100000000000001
6.2 11.0 -4.8
6.1 6.2 -0.10000000000000053
11.0 6.1 4.9
11.1 11.0 0.09999999999999964
11.0 11.1 -0.09999999999999964
16.2 11.0 5.199999999999999
1.0 16.2 -15.2
6.2 1.0 5.2
11.1 6.2 4.8999999999999995
11.0 11.1 -0.09999999999999964
16.1 11.0 5.100000000000001
6.0 16.1 -10.10000000

,t,Eigengeschwindigkeit(m/s),Bbox_Breite,Bbox_Höhe,Bbox_links_unten_y,Abstand(m),v(m/s),v(km/h),a(m/s^2)
0,0.0,22.0,304,226,87,43.5,22.1,79.56,0.0
1,0.1,22.1,276,227,87,43.5,22.1,79.56,0.0
2,0.2,22.1,279,230,87,43.5,22.1,79.56,0.0
3,0.3,22.0,287,236,86,43.0,17.0,61.20,-51.0
4,0.4,22.2,283,233,86,43.0,22.2,79.92,52.0
...,...,...,...,...,...,...,...,...,...
99,9.9,22.0,106,87,157,78.5,17.0,61.20,-102.0
100,10.0,22.1,104,85,159,79.5,32.1,115.56,151.0
101,10.1,22.0,126,77,162,81.0,37.0,133.20,49.0
102,10.2,22.1,101,83,161,80.5,17.1,61.56,-199.0
